In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)



2022-12-18 18:39:38.510437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

class Train_args():

    def __init__(self, learning_rate, weight_decay, epoch):
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.epoch = epoch
         
def train_attack_model(model, train_data, train_target, train_args):

    optimizer = optim.Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)
    
    model.train()
    optimizer.zero_grad()
    output = model(train_data)
    loss = nn.CrossEntropyLoss()(output, train_target.to(torch.long))
    loss.backward(retain_graph=True)
    optimizer.step()

    return model

def attack_evaluation(model, x, y, dev="cpu", extended=False):

    model.eval()

    with torch.no_grad():
        output =  model(x)
        out_target = output.argmax(1, keepdim=True)
        correct = out_target.to(dev).eq(y.to(dev).view_as(out_target.to(dev))).sum().item()
        acc = correct/y.shape[0]

        predicted_positive = output.argmax(1, keepdim=True) == 1
        labeled_positive = y == 1
        tp = predicted_positive.to(dev) * labeled_positive.to(dev).view_as(out_target)
        tp_count = tp.to(dev).sum().item()
        
        if predicted_positive.to(dev).sum().item() != 0:
            pre = tp_count / predicted_positive.to(dev).sum().item()
        else:
            pre = 0
        if labeled_positive.to(dev).sum().item() !=0:
            rec = tp_count / labeled_positive.to(dev).sum().item()
        else:
            rec = 0
    if extended:
        predicted_negative = output.argmax(1, keepdim=True) == 0
        labeled_negative = y == 0
        tn = predicted_negative.to(dev) * labeled_negative.to(dev).view_as(out_target)
        tn_count = tn.to(dev).sum().item()

        fp_count = predicted_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        fn_count = labeled_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        
        return acc, pre, rec, tp_count, tn_count, fp_count, fn_count
    else:
        return acc, pre, rec


In [7]:
raw_data = pd.read_csv('../datasets/dataset_purchase', )
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print(y.nunique())

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.1, random_state=42)
print(X_train.shape, x_shadow.shape)

100
(19732, 600) (177591, 600)


In [8]:
y_target_train[0]

NameError: name 'y_target_train' is not defined

In [ ]:
y_target_train.shape

In [ ]:
# (x_target, y_target), (x_shadow, y_shadow), _, _ = load_nursery(test_set=0.75)

X_train_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:])
y_target_test = np.array(y_train[X_train_size:])

In [17]:
model = algo.LogisticRegression_DPSGD()

In [18]:
params = model.__dict__
p2 = dict(model.__dict__)

In [24]:
model.new = 2
model.__dict__

{'n_classes': 2,
 'alpha': 0.1,
 'max_iter': 100,
 'lambda_': 0.1,
 'tolerance': 1e-06,
 'DP': False,
 'L': 1,
 'C': 1,
 'epsilon': 0.1,
 'delta': 1e-05,
 'new': 2}

In [26]:
p2['train_acc'] ==

{'n_classes': 2,
 'alpha': 0.1,
 'max_iter': 100,
 'lambda_': 0.1,
 'tolerance': 1e-06,
 'DP': False,
 'L': 1,
 'C': 1,
 'epsilon': 0.1,
 'delta': 1e-05}

In [16]:
with open('test_model_param_save.json', 'r') as fp:
    data = json.load(fp)
data

{'n_classes': 2,
 'alpha': 0.1,
 'max_iter': 100,
 'lambda_': 0.1,
 'tolerance': 1e-06,
 'DP': False,
 'L': 1,
 'C': 1,
 'epsilon': 0.1,
 'delta': 1e-05}

In [23]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = 100
model.alpha          = 0.001
model.max_iter       = 100*X_train_size 
model.lambda_        = 1e-3
model.tolerance      = 10e-5
model.DP             = True
model.L              = L
model.C              = C
model.epsilon        = 1
model.delta          = delta

# X,y = model.init_theta(x_target_train, y_target_train)
# model.SGD(X,y)
# model.evaluate(x_target_train, y_target_train)

In [30]:
np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [24]:
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)

The accuracy of the model : 99.6 %
The accuracy of the model : 53.900000000000006 %


In [9]:
tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
if model.DP:
    tm_path += f'_eps{model.epsilon}'
tm_path

NameError: name 'model' is not defined

In [26]:
np.save(tm_path+'_target_model', model.theta)


In [11]:
a = np.load('tm/rs13_lr0.001_iter100_reg1_DPFalse_target_model.npy')

In [12]:
a

array([[ 1.25885028e-03, -3.50712779e-03, -3.16824117e-03, ...,
        -2.35133277e-03,  2.66860303e-03,  2.36332287e-03],
       [ 3.53852769e-44,  3.53852769e-44,  3.53852769e-44, ...,
         3.53852769e-44,  3.53852769e-44,  3.53852769e-44],
       [-2.11209516e-03,  1.15879588e-03, -2.26797171e-03, ...,
        -2.66929941e-03,  7.06403689e-04,  1.72851183e-03],
       ...,
       [-5.93230586e-04,  2.09928426e-03, -4.49838616e-04, ...,
        -5.19930773e-04, -1.61702619e-04,  1.78996053e-04],
       [-3.92912490e-04,  1.71878823e-03, -2.67540245e-04, ...,
        -5.07596203e-04,  1.40477244e-03, -3.68554401e-05],
       [ 1.66235219e-03,  3.19956322e-03, -7.49122078e-04, ...,
         8.07593929e-05, -5.07559610e-04,  1.25629144e-04]])

In [14]:
s_ms = {}
number_of_sms = 10
shadow_size = 50000
shadow_batch_size = int(shadow_size/number_of_sms)

x_shadow_train = np.array(x_shadow[:shadow_size])
y_shadow_train = np.array(y_shadow[:shadow_size])
x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

for i in range(number_of_sms):
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    shadow_model = algo.LogisticRegression_DPSGD()

    shadow_model.n_classes      = 100
    shadow_model.alpha          = 0.001
    shadow_model.max_iter       = 100*shadow_batch_size
    shadow_model.lambda_        = 10e-3
    shadow_model.tolerance      = 10e-5
    shadow_model.DP             = False

    X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
    shadow_model.SGD(X,y)
    print('Shadow model: ', i)
    shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    s_ms[i] = shadow_model

/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_6751/3818404637.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_train = np.array(y_shadow[:shadow_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_6751/3818404637.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])


Shadow model:  0
The accuracy of the model : 93.5 %
Confusion Matrix:
 [[41  0  0 ...  0  0  0]
 [ 0 34  0 ...  0  0  0]
 [ 0  0 28 ...  0  0  0]
 ...
 [ 0  0  0 ... 41  0  0]
 [ 0  0  0 ...  0 56  0]
 [ 0  0  0 ...  0  0 45]]
The accuracy of the model : 44.1 %
Confusion Matrix:
 [[12  0  0 ...  0  0  1]
 [ 0 12  0 ...  0  0  0]
 [ 0  0 13 ...  0  0  0]
 ...
 [ 2  0  0 ... 15  0  0]
 [ 0  0  0 ...  0 28  1]
 [ 0  0  0 ...  1  3  9]]
Shadow model:  1
The accuracy of the model : 94.19999999999999 %
Confusion Matrix:
 [[54  0  0 ...  0  0  0]
 [ 0 23  0 ...  0  0  0]
 [ 0  0 31 ...  0  0  0]
 ...
 [ 3  0  0 ... 19  0  0]
 [ 0  0  0 ...  0 46  0]
 [ 0  0  0 ...  0  0 35]]
The accuracy of the model : 45.300000000000004 %
Confusion Matrix:
 [[29  0  0 ...  0  1  0]
 [ 0 19  0 ...  0  0  0]
 [ 0  0 12 ...  0  0  0]
 ...
 [ 6  0  0 ...  0  0  0]
 [ 1  0  0 ...  0 39  0]
 [ 1  0  0 ...  0  2 12]]
Shadow model:  2
The accuracy of the model : 94.3 %
Confusion Matrix:
 [[34  0  0 ...  0  0  0]
 [ 

In [9]:
# s_ms[0].evaluate(x_shadow_train, y_shadow_train)
# s_ms[0].evaluate(x_shadow_test, y_shadow_test)

In [15]:
shadow_train_pred = []
shadow_test_pred = []

for i in range(number_of_sms): 
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
    shadow_train_pred.append(train_prediciton)
    shadow_test_pred.append(test_prediciton)
    
    

In [16]:
y_shadow_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_train.reshape(-1,1)) #encoode the target values
y_shadow_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_test.reshape(-1,1)) #encoode the target values


In [17]:
sh_train_pred = np.concatenate(shadow_train_pred)
sh_test_pred = np.concatenate(shadow_test_pred)

# members
labels = np.ones(sh_train_pred.shape[0])
# non-members
test_labels = np.zeros(sh_test_pred.shape[0])

x_1 = np.concatenate((sh_train_pred, sh_test_pred))
x_2 = np.concatenate((y_shadow_train_ohe, y_shadow_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))


In [18]:
attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)

In [92]:
# attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
# attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-6, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)

In [28]:
shadow_model.n_classes      = 100
shadow_model.alpha          = 0.001
shadow_model.max_iter       = 100*shadow_batch_size
shadow_model.lambda_        = 10e-3
shadow_model.tolerance      = 10e-5
shadow_model.DP             = False

In [60]:
import os

In [61]:
sh_path = f'mia/shms{number_of_sms}_shtrsize{shadow_size}_shlr{shadow_model.alpha}_shiter{int(shadow_model.max_iter/shadow_batch_size)}_shreg{shadow_model.lambda_}/'      
os.mkdir(sh_path)

torch.save(attack_train_data, sh_path+'attack_train_data.pt')
torch.save(attack_train_target, sh_path+'attack_train_target.pt')

at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
torch.save(attack_model, at_path)


In [74]:
train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
train_acc, train_pre, train_rec 

(0.59217, 0.5930014328093153, 0.5877)

In [75]:
test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

(0.6260389215487533, 0.5935737236701178, 0.8313)

In [ ]:
# MI optimization

In [ ]:
ams

In [87]:
rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

epo = [200]
wd = [1e-5, 1e-6]
ams = {}
for ep in epo:
    for w_d in wd:
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=ep)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        for epoch in range(attack_train_args.epoch):

            attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
            ams[(l_r,w_d)] = attack_model
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        torch.save(attack_model, at_path)


In [91]:
epo = [200,500]
wd = [1e-5, 1e-6]

for ep in epo:
    for w_d in wd:
        print(ep, w_d)
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=500)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        attack_model = torch.load(at_path)
        
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print('Train: ', train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print('Test: ' , test_acc, test_pre, test_rec)


200 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
200 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99
500 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
500 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99


In [ ]:
for l_r in lr:
    for w_d in wd:
        ams[(l_r,w_d)]
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print("Train acc pre rec: ", train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print("Test acc pre rec: ", test_acc, test_pre, test_rec)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
shadow_size = 20000
shadow_clf = LogisticRegression(random_state=1).fit(x_shadow[:shadow_size], y_shadow[:shadow_size])

In [ ]:
print(shadow_clf.score(x_shadow[:shadow_size], y_shadow[:shadow_size]))
print(shadow_clf.score(x_shadow[shadow_size:shadow_size*2], y_shadow[shadow_size:shadow_size*2]))

In [ ]:
shadow_train_pred = shadow_clf.predict_proba(x_shadow[:shadow_size])
shadow_test_pred = shadow_clf.predict_proba(x_shadow[shadow_size:shadow_size*2])
y_shadow_train = y_shadow[:shadow_size]
y_shadow_test = y_shadow[shadow_size:shadow_size*2]

In [ ]:
# members
labels = np.ones(shadow_train_pred.shape[0])
# non-members
test_labels = np.zeros(shadow_test_pred.shape[0])

x_1 = np.concatenate((shadow_train_pred, shadow_test_pred))
x_2 = np.concatenate((y_shadow_train, y_shadow_test)).reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = np.array(df.drop(['a_target'], axis=1))
attack_train_target = np.array(df['a_target'])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [ ]:
train_acc, train_pre, train_rec 

In [67]:
train_pred = model.predict(x_target_train, y_target_train)
test_pred = model.predict(x_target_test, y_target_test)

y_target_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_train.reshape(-1,1)) #encoode the target values
y_target_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_test.reshape(-1,1)) #encoode the target values
    
# members
labels = np.ones(train_pred.shape[0])
# non-members
test_labels = np.zeros(test_pred.shape[0])

x_1 = np.concatenate((train_pred, test_pred))
x_2 = np.concatenate((y_target_train_ohe, y_target_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_test_data = np.concatenate((x_1,x_2),axis=1)
attack_test_target = y_new
df = pd.DataFrame(attack_test_data)
df['a_target'] = attack_test_target
df = df.sample(frac = 1)

attack_test_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_test_target = torch.tensor(np.array(df['a_target']))


In [ ]:
test_acc, test_pre, test_rec = attack_evaluation(a_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

In [14]:
a_model = Net_attack(h_neurons=64, do=0, input_size=100)
a_model = torch.load('attack_model') 

In [ ]:

rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

lr = [0.001]
wd = [1e-4, 1e-6]
tms = {}
for l_r in lr:
    for w_d in wd:
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = 100
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size 
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.epsilon

        X,y = model.init_theta(x_target_train, y_target_train)
        model.SGD(X,y)
        model.evaluate(x_target_train, y_target_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import algo
# import attack

# from torch import nn,optim
import torch

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
        

raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")

for L in [1,10,100]:
    for epsilon in np.arange(0,1,0.1):
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = n_classes
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.L              = L
        model.epsilon        = epsilon


        X,y = model.init_theta(x_target_train, y_target_train)
        model.train(X,y)
        model.evaluate(x_target_train, y_target_train, acc=True)
        model.evaluate(x_target_test, y_target_test, acc=True)

        tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
        if model.DP:
            tm_path += f'_eps{model.epsilon}_L{model.L}'
        np.save(tm_path+'_target_model', model.theta)

        print(tm_path)

    

In [ ]:
#Shadow models
# s_ms = {}
# number_of_sms = 10
# shadow_size = 50000
# shadow_batch_size = int(shadow_size/number_of_sms)

# x_shadow_train = np.array(x_shadow[:shadow_size])
# y_shadow_train = np.array(y_shadow[:shadow_size])
# x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
# y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

# attack.train_shadow_models(number_of_sms,)

# for i in range(number_of_sms):  
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     shadow_model = algo.LogisticRegression_DPSGD()

#     shadow_model.n_classes      = n_classes
#     shadow_model.alpha          = 0.001
#     shadow_model.max_iter       = 100*shadow_batch_size
#     shadow_model.lambda_        = 10e-3
#     shadow_model.tolerance      = 10e-5
#     shadow_model.DP             = False

#     X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
#     shadow_model.SGD(X,y)
#     print('Shadow model: ', i)
#     shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
#     s_ms[i] = shadow_model

# #Attack model

# shadow_train_pred = []
# shadow_test_pred = []

# for i in range(number_of_sms): 
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
#     shadow_train_pred.append(train_prediciton)
#     shadow_test_pred.append(test_prediciton)
    

In [10]:
1/10000 == 1e-5


False

In [3]:
np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])